In [3]:
import os

In [4]:
%pwd


'e:\\all ML\\DeepLearning\\KidneyDiseaseClassification-DL-MLflow-DVC\\research'

In [5]:
os.chdir("../") #changing the directory going back
%pwd


'e:\\all ML\\DeepLearning\\KidneyDiseaseClassification-DL-MLflow-DVC'

In [6]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir :Path
    source_url :str 
    local_data_file :Path
    unzip_dir :Path

In [7]:
from src.cnnKDClassification.constants import * 
from src.cnnKDClassification.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )        
        return data_ingestion_config

In [9]:
import os 
import urllib.request as request
import zipfile
import gdown
from src.cnnKDClassification.utils.common import get_size
from src.cnnKDClassification import logger


In [10]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        
        
    def download_file(self)-> str:
        '''Fetch the data from the url
        
        '''
        try: 
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"downloading the data form {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"downloaded the data form {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e 
        
    def extract_zip_file(self):
        """zip file path:str
        extract the zip file into the data directory 
        Fun return data
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-19 13:26:17,090 :INFO: yaml file : <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> loaded successfully]


[2023-10-19 13:26:17,097 :INFO: yaml file : <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2023-10-19 13:26:17,101 :INFO: created directory at :artifacts]
[2023-10-19 13:26:17,104 :INFO: created directory at :artifacts/data_ingestion]
[2023-10-19 13:26:17,107 :INFO: downloading the data form https://drive.google.com/file/d/1-iaNzdgL8LwCm68ymGtYCWGJgfWeBsAH/view?usp=share_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1-iaNzdgL8LwCm68ymGtYCWGJgfWeBsAH
From (redirected): https://drive.google.com/uc?/export=download&id=1-iaNzdgL8LwCm68ymGtYCWGJgfWeBsAH&confirm=t&uuid=6e92a5e3-f72e-4e1c-a0a8-50562d670b24
To: e:\all ML\DeepLearning\KidneyDiseaseClassification-DL-MLflow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [01:28<00:00, 651kB/s] 

[2023-10-19 13:27:49,656 :INFO: downloaded the data form https://drive.google.com/file/d/1-iaNzdgL8LwCm68ymGtYCWGJgfWeBsAH/view?usp=share_link into file artifacts/data_ingestion/data.zip]
